In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import random

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Load the Iris dataset
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for training and testing sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [4]:
class IrisNet(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64, output_dim=3):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)  # Specify out_features for fc2
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

# Initialize the model
model = IrisNet()


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [6]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    # Periodic evaluation
    if (epoch + 1) % 100 == 0:
        model.eval()
        with torch.no_grad():
            correct, total = 0, 0
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                total += y_batch.size(0)
                correct += (predicted == y_batch).sum().item()
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')


Epoch [100/1000], Loss: 0.8049, Accuracy: 80.00%
Epoch [200/1000], Loss: 0.7697, Accuracy: 93.33%
Epoch [300/1000], Loss: 0.6994, Accuracy: 100.00%
Epoch [400/1000], Loss: 0.5829, Accuracy: 100.00%
Epoch [500/1000], Loss: 0.5590, Accuracy: 100.00%
Epoch [600/1000], Loss: 0.5689, Accuracy: 100.00%
Epoch [700/1000], Loss: 0.5730, Accuracy: 100.00%
Epoch [800/1000], Loss: 0.6080, Accuracy: 100.00%
Epoch [900/1000], Loss: 0.5563, Accuracy: 100.00%
Epoch [1000/1000], Loss: 0.5550, Accuracy: 100.00%


In [7]:
torch.save(model.state_dict(), 'iris_model.pth')


In [9]:
pip install tensorboard


   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB 146.3 kB/s eta 0:00:38
   ---------------------------------------- 0.0/5.5 MB 164.3 kB/s eta 0:00:34
   ---------------------------------------- 0.1/5.5 MB 218.8 kB/s eta 0:00:25
    --------------------------------------- 0.1/5.5 MB 231.8 kB/s eta 0:00:24
    --------------------------------------- 0.1/5.5 MB 312.2 kB/s eta 0:00:18
    --------------------------------------- 0.1/5.5 MB 313.8 kB/s eta 0:00:18
   - -------------------------------------- 0.2/5.5 MB 374.9 kB/s eta 0:00:15
   - -------------------------------------- 0.2/5.5 MB 402.0 kB/s eta 0:00:14
   - -------------------------------------- 0.2/5.5 MB 437.4 kB/s eta 0:00:13
   -- ------------------------------------- 0.3/5.5 MB 473.7 kB/s eta 0:00:12
   -- -------

In [1]:
# Import required modules
%run train.py


Epoch [100/1000], Loss: 0.7582, Accuracy: 83.33%
Epoch [200/1000], Loss: 0.6645, Accuracy: 96.67%
Epoch [300/1000], Loss: 0.6698, Accuracy: 100.00%
Epoch [400/1000], Loss: 0.5580, Accuracy: 100.00%
Epoch [500/1000], Loss: 0.5645, Accuracy: 100.00%
Epoch [600/1000], Loss: 0.5732, Accuracy: 100.00%
Epoch [700/1000], Loss: 0.5536, Accuracy: 100.00%
Epoch [800/1000], Loss: 0.5565, Accuracy: 100.00%
Epoch [900/1000], Loss: 0.5936, Accuracy: 100.00%
Epoch [1000/1000], Loss: 0.5522, Accuracy: 100.00%
